In [1]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
import pandas as pd
from collections import OrderedDict
from datetime import date
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import fiona
import shapely
import datetime

In [2]:
"""
AOI Parameters
"""

# bbox
bbox = (-4.860230,47.428380,-0.982056,48.897512)
bbox_poly = shapely.geometry.box(*bbox, ccw=True)


# start-end times
start = datetime.datetime(2018,4,1)
end = datetime.datetime(2018,9,30)

In [3]:
# define tiles
tiles = ['30UUU', '30UVV', '30UVU', '30TVT', '30UWU', '30TWT', '30UXU']


# set up API
user = "sdonike" ####### ## change this!
pw =  "Iha0504G$" ######## ## change this!
api = SentinelAPI(user, pw, api_url='https://scihub.copernicus.eu/dhus', show_progressbars= True,timeout=100)


In [4]:
# set query params
query_kwargs = {
        'platformname': 'Sentinel-2',
        #'producttype': 'S2MSI2Ap',
        #'processinglevel': 'Level-1C',
        'date': ('20210401', '20210930'),
        'cloudcoverpercentage': (0, 20),
         'limit': 999} 

In [5]:
#products_ls = []
footprints = []

products = OrderedDict()
for tile in tiles:
    products = OrderedDict()
    kw = query_kwargs.copy()
    kw['tileid'] = tile  
    pp = api.query(**kw)
    products.update(pp)
    #products_ls.append(products)
    print("Tile:",tile,"\t - Products:",len(products))

    
    products_df = api.to_dataframe(products)
    products_df_sorted = products_df.sort_values(['size'],ascending = [False])
    #products_df_sorted = products_df_sorted.head (1)

    #print(products_df_sorted['size'])
    #print(products_df_sorted['title'])
    
    fp = []
    for i in products_df_sorted["footprint"]:
        fp.append(i)
        
    #tmp_ls = [products_df_sorted["title"],products_df_sorted["tileid"],products_df_sorted["footprint"],products_df_sorted["datatakesensingstart"]]
    
    footprints.append(fp)
    
    
    #download data
    # api.download_all(products_df_sorted.index, directory_path= r'C:\Users\EQuilibrium\PyBegining\downloader\Downld_Sen',max_attempts=2, checksum = True )
    #api.download(products_df_sorted.index, directory_path= r'C:\Users\EQuilibrium\PyBegining\downloader\Downld_Sen', checksum = True )
    #print(products_df_sorted)
# flatten list
footprints = [item for sublist in footprints for item in sublist]


Tile: 30UUU 	 - Products: 34
Tile: 30UVV 	 - Products: 27
Tile: 30UVU 	 - Products: 15
Tile: 30TVT 	 - Products: 27
Tile: 30UWU 	 - Products: 25
Tile: 30TWT 	 - Products: 35
Tile: 30UXU 	 - Products: 14


In [6]:
footprints_df = pd.DataFrame(footprints,columns=["geometry"])
footprints_df

,geometry
0,"MULTIPOLYGON (((-4.5203247 47.753511601172576,..."
1,"MULTIPOLYGON (((-4.5111084 47.75366711311768, ..."
2,"MULTIPOLYGON (((-4.51593 47.753585718252026, -..."
3,"MULTIPOLYGON (((-4.5126038 47.753641969921134,..."
4,"MULTIPOLYGON (((-4.505188 47.75376698671734, -..."
...,...
172,"MULTIPOLYGON (((-0.43804932 47.73529773109481,..."
173,MULTIPOLYGON (((-0.44329834 47.735392341874515...
174,MULTIPOLYGON (((-0.43945312 47.735323263511006...
175,MULTIPOLYGON (((-0.43118286 47.735174189104235...


In [7]:
footprints_df['geometry'] = gpd.GeoSeries.from_wkt(footprints_df['geometry'])
gdf = gpd.GeoDataFrame(footprints_df, geometry='geometry')


In [8]:
pwd

'/home/simon/CDE_UBS/thesis/data_collection/sen2'

In [11]:
gdf.to_file('sen2_tiles/qgis_files/footprints.shp')  

DriverIOError: Failed to create file sen2_tiles/qgis_files/footprints.shp: No such file or directory

In [10]:
pwd

'/home/simon/CDE_UBS/thesis/data_collection/sen2'